In [ ]:
using ZipFile, SExpressions

In [ ]:
out = download("https://github.com/thelmuth/Clojush/archive/refs/tags/psb2-v1.0.zip")

r = ZipFile.Reader(out);

In [ ]:
r = ZipFile.Reader(out)
instructions = filter(x -> contains(x.name, "instruction") && contains(x.name, ".clj"), r.files)
instruction_set = Dict()

for i in instructions
    name = split(i.name, "/")[end]
    contents = String(read(i))
    contents = replace(
        contents,
        r"\#\{(?<setitems>.+)\}" => s"(hash-set \g<setitems>)", # Replace #{...} with (hash-set ...)
        r"#\_" => "", # Replace #_ (comment) with nothing
        r"#\(" => "(ANONYMOUS ", # Replace #"..." (regex in Clojure) with r"..." (regex in Julia)
        r"#" => "r",
        "\\" => "\\\\",
        ";(update-in [:gtm :position] inc)" => "",
    )

    try
        s = SExpressions.parseall(contents)
        instruction_set[name] = s.cdr
    catch err
        println(">>> ERROR parsing $name: $err")
    end
end

In [ ]:
function make_instruction_table(instruction_set)
    """
    Recursively get the .cdr of each instruction in the instruction set.
    """
    instructions = Dict()
    if isa(instruction_set, Nil)
        return instructions
    end
    if instruction_set.car.car == Symbol("define-registered")
        instructions[instruction_set.car.cdr.car] = instruction_set.car.cdr.cdr
        instructions = merge(instructions, make_instruction_table(instruction_set.cdr))
    else
        instructions = merge(instructions, make_instruction_table(instruction_set.cdr))
    end

    return instructions
end

In [ ]:
flattened_instruction_sets = Dict(k => make_instruction_table(v) for (k, v) in instruction_set)

In [ ]:
for (k, v) in flattened_instruction_sets
    for (k2, v2) in v
        println("$k.$k2")
    end
end